**Notes**:
This notebook prepares the [example sleep data - sleep.csv](https://github.com/LSYS/pyforestplot/blob/main/examples/data/sleep.csv).

The resulting output csv file ([sleep.csv](https://github.com/LSYS/pyforestplot/blob/main/examples/data/sleep.csv)) that indicates how certain individual characteristics correlates to the amount of sleep an one gets per week.
Rows are the variables correlating with sleep. Columns included the computed pearson correlation coefficient, sample size, p-value, confidence interval (95%), etc.
The `pingouin` is used to compute correlations.

**Raw src**:
* `sleep75.csv` (/wooldridge/sleep75) from https://vincentarelbundock.github.io/Rdatasets/articles/data.html
* See https://rdrr.io/cran/wooldridge/man/sleep75.html for variable labels to the variables in `sleep75.csv`.



**Requirements**: Mainly `pingouin`. See first cell of imports for requirements

In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import warnings
warnings.filterwarnings('ignore')

_url = "https://vincentarelbundock.github.io/Rdatasets/csv/wooldridge/sleep75.csv"
drop_var = ['case', 'leis1', 'leis2', 'leis3']
df = (pd.read_csv(_url, index_col=0)
      .drop(drop_var, axis=1)
     )
df.head(3)

,age,black,clerical,construc,educ,earns74,gdhlth,inlf,smsa,lhrwage,...,spwrk75,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq
1,32,0,0.0,0.0,12,0,0,1,0,1.955861,...,0,3438,0,3438,0,14,0,13,7.070004,1024
2,31,0,0.0,0.0,14,9500,1,1,0,0.357674,...,0,5020,0,5020,0,11,0,0,1.429999,961
3,44,0,0.0,0.0,17,42500,1,1,1,3.021887,...,1,2815,0,2815,0,21,0,0,20.529997,1936


In [2]:
# Prep variable lablels (fold cell)
# varlabels: http://fmwww.bc.edu/ec-p/data/wooldridge/sleep75.des
df_label = (pd.read_csv('data/sleep75-des.csv')
            .assign(label=lambda df: df['des'].str.encode('ascii', 'ignore').str.decode('ascii'))
            .drop(['des'], axis=1)
            .set_index('var')
            .drop(drop_var)
            .reset_index()
           )

df_label.head(3)

,var,group,label
0,age,age,in years
1,black,other factors,=1 if black
2,clerical,occupation,=1 if clerical worker


In [3]:
# Compute correlations
df_corr = (pg.pairwise_corr(df)
           .rename(columns={'p-unc': 'p-val'})
           .query('Y=="sleep"|X=="sleep"')
           .assign(var=lambda df: df['X'])
           .assign(var=lambda df: np.where(df['var']=="sleep", df['Y'], df['var']))
           .drop(["Y", "X", "method", "alternative"], axis=1)
           .assign(
               hl=lambda df: [float(ci[1]) for ci in df['CI95%']],
               ll=lambda df: [float(ci[0]) for ci in df['CI95%']],
               moerror=lambda df: df['hl'] - df['r'],
               power=lambda df: df.power.round(decimals=2),
               n=lambda df: df.n.map(str)
           )
           # Get labels
           .merge(df_label, how='left', on='var', validate='1:1')
           .reset_index(drop=True)
          )
df_corr

,n,r,CI95%,p-val,BF10,power,var,hl,ll,moerror,group,label
0,706,0.090373,"[0.02, 0.16]",1.630887e-02,0.839,0.67,age,0.16,0.02,0.069627,age,in years
1,706,-0.027057,"[-0.1, 0.05]",4.728889e-01,0.061,0.11,black,0.05,-0.10,0.077057,other factors,=1 if black
2,706,0.048081,"[-0.03, 0.12]",2.019484e-01,0.106,0.25,clerical,0.12,-0.03,0.071919,occupation,=1 if clerical worker
3,706,0.041229,"[-0.03, 0.11]",2.739475e-01,0.086,0.19,construc,0.11,-0.03,0.068771,occupation,=1 if construction worker
4,706,-0.095004,"[-0.17, -0.02]",1.155151e-02,1.137,0.72,educ,-0.02,-0.17,0.075004,labor factors,years of schooling
5,706,-0.076890,"[-0.15, -0.0]",4.110934e-02,0.378,0.53,earns74,-0.00,-0.15,0.076890,labor factors,"total earnings, 1974"
6,706,-0.102825,"[-0.18, -0.03]",6.246660e-03,1.967,0.78,gdhlth,-0.03,-0.18,0.072825,health factors,=1 if in good or excel. health
7,706,-0.027126,"[-0.1, 0.05]",4.717698e-01,0.061,0.11,inlf,0.05,-0.10,0.077126,labor factors,=1 if in labor force
8,706,-0.066997,"[-0.14, 0.01]",7.524015e-02,0.229,0.43,smsa,0.01,-0.14,0.076997,area of residence,=1 if live in smsa
9,532,-0.067197,"[-0.15, 0.02]",1.216222e-01,0.179,0.34,lhrwage,0.02,-0.15,0.087197,labor factors,log hourly wage


In [4]:
df_corr.to_csv('data/sleep-untruncated.csv', index=False)

_drop = ['earns74', 'inlf', 'lothinc', 'workscnd', 'lhrwage', 'worknrm', 
         'spwrk75', 'marr', 'black', 'agesq', 'union', 'exper', 'rlxall', 'slpnaps']
df_corr.query('var not in @_drop').to_csv('data/sleep.csv', index=False)

In [5]:
_cols = ['var', 'r', 'moerror', 'label', 'group', 'll', 'hl', 'n', 'power', 'p-val']
print(df_corr[_cols].head(3).to_markdown())

|    | var      |          r |   moerror | label                 | group         |    ll |   hl |   n |   power |     p-val |
|---:|:---------|-----------:|----------:|:----------------------|:--------------|------:|-----:|----:|--------:|----------:|
|  0 | age      |  0.0903729 | 0.0696271 | in years              | age           |  0.02 | 0.16 | 706 |    0.67 | 0.0163089 |
|  1 | black    | -0.0270573 | 0.0770573 | =1 if black           | other factors | -0.1  | 0.05 | 706 |    0.11 | 0.472889  |
|  2 | clerical |  0.0480811 | 0.0719189 | =1 if clerical worker | occupation    | -0.03 | 0.12 | 706 |    0.25 | 0.201948  |
